# Load the pymoo result object saved with pickle.
ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

## Load result object
Place a pymoo result object file saved with pickle and execute the following cell.
Replace 'res_Cogent.pkl' with your result file saved with pickle.

In [2]:
with open('res_bc_Cogent_smsemoa.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_smsemoa.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_smsemoa.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_smsemoa.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_smsemoa.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_smsemoa.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_smsemoa.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_smsemoa.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
# with open('res_bc_Kdl_smsemoa.pkl','rb') as f_Kdl:
#     res_Kdl = pickle.load(f_Kdl)
with open('res_bc_Internode_smsemoa.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_smsemoa.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_smsemoa.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_smsemoa.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

## Check whether all results meet the constranins
- The number of controller >= 2
- The number of Atomix node  >= 3
- The number of controller should be a integer number
- The number of Atomix node should be a integer number

In [3]:
# F = res.F
# print(F[np.argsort(F[:, 2])])
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
# F_Kdl = res_Kdl.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

# Check conditions
c1_Cogent = F_Cogent[:, 0] >= 2  # the number of controller >= 2
c2_Cogent = F_Cogent[:, 1] >= 3  # the number of Atomix node  >= 3
c3_Cogent = F_Cogent[:, 0] % 1 == 0 # the number of controller should be a integer number
c4_Cogent = F_Cogent[:, 1] % 1 == 0 # the number of Atomix node should be a integer number
c1_UsCarrier = F_UsCarrier[:, 0] >= 2
c2_UsCarrier = F_UsCarrier[:, 1] >= 3
c3_UsCarrier = F_UsCarrier[:, 0] % 1 == 0 
c4_UsCarrier = F_UsCarrier[:, 1] % 1 == 0 
c1_HiberniaGlobal = F_HiberniaGlobal[:, 0] >= 2
c2_HiberniaGlobal = F_HiberniaGlobal[:, 1] >= 3
c3_HiberniaGlobal = F_HiberniaGlobal[:, 0] % 1 == 0 
c4_HiberniaGlobal = F_HiberniaGlobal[:, 1] % 1 == 0 
c1_Colt = F_Colt[:, 0] >= 2
c2_Colt = F_Colt[:, 1] >= 3
c3_Colt = F_Colt[:, 0] % 1 == 0 
c4_Colt = F_Colt[:, 1] % 1 == 0 
c1_Funet = F_Funet[:, 0] >= 2
c2_Funet = F_Funet[:, 1] >= 3
c3_Funet = F_Funet[:, 0] % 1 == 0 
c4_Funet = F_Funet[:, 1] % 1 == 0 
c1_Abvt = F_Abvt[:, 0] >= 2
c2_Abvt = F_Abvt[:, 1] >= 3
c3_Abvt = F_Abvt[:, 0] % 1 == 0 
c4_Abvt = F_Abvt[:, 1] % 1 == 0 
c1_Intellifiber = F_Intellifiber[:, 0] >= 2
c2_Intellifiber = F_Intellifiber[:, 1] >= 3
c3_Intellifiber = F_Intellifiber[:, 0] % 1 == 0 
c4_Intellifiber = F_Intellifiber[:, 1] % 1 == 0 
c1_TataNld = F_TataNld[:, 0] >= 2
c2_TataNld = F_TataNld[:, 1] >= 3
c3_TataNld = F_TataNld[:, 0] % 1 == 0 
c4_TataNld = F_TataNld[:, 1] % 1 == 0 
# c1_Kdl = F_Kdl[:, 0] >= 2
# c2_Kdl = F_Kdl[:, 1] >= 3
# c3_Kdl = F_Kdl[:, 0] % 1 == 0 
# c4_Kdl = F_Kdl[:, 1] % 1 == 0 
c1_Internode = F_Internode[:, 0] >= 2
c2_Internode = F_Internode[:, 1] >= 3
c3_Internode = F_Internode[:, 0] % 1 == 0 
c4_Internode = F_Internode[:, 1] % 1 == 0 
c1_Missouri = F_Missouri[:, 0] >= 2
c2_Missouri = F_Missouri[:, 1] >= 3
c3_Missouri = F_Missouri[:, 0] % 1 == 0 
c4_Missouri = F_Missouri[:, 1] % 1 == 0 
c1_Ion = F_Ion[:, 0] >= 2
c2_Ion = F_Ion[:, 1] >= 3
c3_Ion = F_Ion[:, 0] % 1 == 0 
c4_Ion = F_Ion[:, 1] % 1 == 0 
c1_Palmetto = F_Palmetto[:, 0] >= 2
c2_Palmetto = F_Palmetto[:, 1] >= 3
c3_Palmetto = F_Palmetto[:, 0] % 1 == 0 
c4_Palmetto = F_Palmetto[:, 1] % 1 == 0 

if np.all(c1_Cogent) and np.all(c2_Cogent) and np.all(c3_Cogent) and np.all(c4_Cogent) and np.all(c1_UsCarrier) and np.all(c2_UsCarrier) and np.all(c3_UsCarrier) and np.all(c4_UsCarrier) and np.all(c1_HiberniaGlobal) and np.all(c2_HiberniaGlobal) and np.all(c3_HiberniaGlobal) and np.all(c4_HiberniaGlobal) and np.all(c1_Colt) and np.all(c2_Colt) and np.all(c3_Colt) and np.all(c4_Colt)  and np.all(c1_Funet) and np.all(c2_Funet) and np.all(c3_Funet) and np.all(c4_Funet) and np.all(c1_Abvt) and np.all(c2_Abvt) and np.all(c3_Abvt) and np.all(c4_Abvt) and np.all(c1_Intellifiber) and np.all(c2_Intellifiber) and np.all(c3_Intellifiber) and np.all(c4_Intellifiber) and np.all(c1_TataNld) and np.all(c2_TataNld) and np.all(c3_TataNld) and np.all(c4_TataNld) and np.all(c1_Internode) and np.all(c2_Internode) and np.all(c3_Internode) and np.all(c4_Internode) and np.all(c1_Missouri) and np.all(c2_Missouri) and np.all(c3_Missouri) and np.all(c4_Missouri) and np.all(c1_Ion) and np.all(c2_Ion) and np.all(c3_Ion) and np.all(c4_Ion) and np.all(c1_Palmetto) and np.all(c2_Palmetto) and np.all(c3_Palmetto) and np.all(c4_Palmetto):
    print(True)
else: 
    print(False)

True
